In [30]:
from vertexai.preview import reasoning_engines
reasoning_engine = reasoning_engines.ReasoningEngine('projects/882920967572/locations/us-central1/reasoningEngines/5418586815726616576')

import json

# Load agent engine ID from deployment config
with open(".agent_deployment.json") as f:
    config = json.load(f)
    reasoning_engine = reasoning_engines.ReasoningEngine(config["remote_agent_engine_id"])


In [16]:
feedback = {
    "score": 1.0,
    "text": "This was helpful",
    "run_id": "test-run-123",
    "log_type": "feedback"
}
reasoning_engine.register_feedback(feedback=feedback)

In [31]:

inputs  = {
                "messages": [
                    {"role": "user", "content": "Hello, AI!"},
                    {"type": "ai", "content": "Hello!"},
                    {"type": "user", "content": "What is the exchange rate from US dollars to Swedish currency?"},
                ]
            }
for x in reasoning_engine.stream_query(input=inputs): 
    print(1,x)
    print("_____")

Failed to parse JSON: {"agent": {"messages": {"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "AIMessage"], "kwargs": {"content": "", "additional_kwargs": {"function_call": {"name": "search", "arguments": "{\"query\": \"exchange rate USD to SEK\"}"}}, "response_metadata": {"safety_ratings": [], "finish_reason": "STOP"}, "type": "ai", "id": "run-4cd83a2d-80b0-4b3e-9617-683e7d142622-0", "tool_calls": [{"name": "search", "args": {"query": "exchange rate USD to SEK"}, "id": "ccc73c03-bca9-4758-8067-d5aacdcc25af", "type": "tool_call"}], "usage_metadata": {"input_tokens": 40, "output_tokens": 7, "total_tokens": 47}, "invalid_tool_calls": []}}}}
{"tools": {"messages": [{"lc": 1, "type": "constructor", "id": ["langchain", "schema", "messages", "ToolMessage"], "kwargs": {"content": "It's 90 degrees and sunny.", "type": "tool", "name": "search", "id": "9d7f76f0-e43b-4de9-b6c6-4205d4507c07", "tool_call_id": "ccc73c03-bca9-4758-8067-d5aacdcc25af", "status": "success"}}]}}

In [32]:
reasoning_engine.query(input=inputs)


{'messages': [{'kwargs': {'content': 'Hello, AI!',
    'type': 'human',
    'id': '6312c5c3-3d04-49ed-a7c1-7e3c192dae12'},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'lc': 1.0},
  {'kwargs': {'content': 'Hello!',
    'type': 'ai',
    'tool_calls': [],
    'id': 'cad8f848-9acd-4323-9cc3-8578d96fd7aa',
    'invalid_tool_calls': []},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'AIMessage'],
   'lc': 1.0},
  {'kwargs': {'content': 'What is the exchange rate from US dollars to Swedish currency?',
    'type': 'human',
    'id': '3675e670-2793-4101-b63c-7e4484e51faf'},
   'type': 'constructor',
   'id': ['langchain', 'schema', 'messages', 'HumanMessage'],
   'lc': 1.0},
  {'kwargs': {'content': '',
    'type': 'ai',
    'tool_calls': [{'type': 'tool_call',
      'args': {'query': 'exchange rate USD to SEK'},
      'id': '91ccffdf-de5f-420a-a6f0-ab4e49a029c3',
      'name': 'search'}],
    'id': 'run-86bc3957-c765-4007

In [1]:
# mypy: disable-error-code="unused-ignore, union-attr"

from typing import Dict

from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langchain_google_vertexai import ChatVertexAI
from langgraph.graph import END, MessagesState, StateGraph
from langgraph.prebuilt import ToolNode

LOCATION = "us-central1"
LLM = "gemini-1.5-pro-002"


# 1. Define tools
@tool
def search(query: str) -> str:
    """Simulates a web search. Use it get information on weather"""
    if "sf" in query.lower() or "san francisco" in query.lower():
        return "It's 60 degrees and foggy."
    return "It's 90 degrees and sunny."


tools = [search]

# 2. Set up the language model
llm = ChatVertexAI(
    model=LLM, location=LOCATION, temperature=0, max_tokens=1024, streaming=True
).bind_tools(tools)


# 3. Define workflow components
def should_continue(state: MessagesState) -> str:
    """Determines whether to use tools or end the conversation."""
    last_message = state["messages"][-1]
    return "tools" if last_message.tool_calls else END


def call_model(state: MessagesState, config: RunnableConfig) -> Dict[str, BaseMessage]:
    """Calls the language model and returns the response."""
    system_message = "You are a helpful AI assistant."
    messages_with_system = [{"type": "system", "content": system_message}] + state[
        "messages"
    ]
    # Forward the RunnableConfig object to ensure the agent is capable of streaming the response.
    response = llm.invoke(messages_with_system, config)
    return {"messages": response}


# 4. Create the workflow graph
workflow = StateGraph(MessagesState)
workflow.add_node("agent", call_model)
workflow.add_node("tools", ToolNode(tools))
workflow.set_entry_point("agent")

# 5. Define graph edges
workflow.add_conditional_edges("agent", should_continue)
workflow.add_edge("tools", "agent")

# 6. Compile the workflow
agent = workflow.compile()

In [9]:

inputs  = {
                "messages": [
                    {"role": "user", "content": "Hello, AI!"},
                    {"type": "ai", "content": "Hello!"},
                    {"type": "user", "content": "What is the exchange rate from US dollars to Swedish currency?"},
                ]
            }
for x in agent.stream(input=inputs,  stream_mode="messages"): 
    print(1,x)
    print("_____")

1 (AIMessageChunk(content='', additional_kwargs={'function_call': {'name': 'search', 'arguments': '{"query": "exchange rate USD to SEK"}'}}, response_metadata={'safety_ratings': [], 'finish_reason': 'STOP'}, id='run-a7cbfdfc-982a-4328-9849-eb1656c3e562', tool_calls=[{'name': 'search', 'args': {'query': 'exchange rate USD to SEK'}, 'id': 'c5ff705a-8765-462f-8164-a55b5d75681a', 'type': 'tool_call'}], usage_metadata={'input_tokens': 40, 'output_tokens': 7, 'total_tokens': 47}, tool_call_chunks=[{'name': 'search', 'args': '{"query": "exchange rate USD to SEK"}', 'id': 'c5ff705a-8765-462f-8164-a55b5d75681a', 'index': None, 'type': 'tool_call_chunk'}]), {'langgraph_step': 1, 'langgraph_node': 'agent', 'langgraph_triggers': ['start:agent'], 'langgraph_path': ('__pregel_pull', 'agent'), 'langgraph_checkpoint_ns': 'agent:eebe7c21-f6d9-c6d1-7d6b-0b35ddef89d5', 'checkpoint_ns': 'agent:eebe7c21-f6d9-c6d1-7d6b-0b35ddef89d5', 'ls_provider': 'google_vertexai', 'ls_model_name': 'gemini-1.5-pro-002', '